# Simulations
In this iPython notebook, we run the double auction version of the model. 

In [1]:
from ipywidgets import widgets, interact
from stockmarket import baselineCDA
import sqlite3
#from tqdm import *
from stockmarket.database import create_CDA_tables, record_state_variables

In [2]:
NRUNS = 10
EXPERIMENT_ID = 1
EXPERIMENT = 0
SIMULATIONTIME = 100

In [3]:
conn = sqlite3.connect('dataBaseCDAuction.sqlite')
cur = conn.cursor()

In [4]:
for seed in range(NRUNS):
    create_CDA_tables(cur, seed)
    agents, firms, stocks, order_books = baselineCDA.stockMarketSimulation(seed=seed, simulation_time=SIMULATIONTIME,
                                                                           amount_fundamentalists=40, 
                                                                           amount_chartists=40, amount_firms=1,
                                                                           initial_money=(100, 200), 
                                                                           initial_bid_ask=(1,1), 
                                                                           initial_memory=(3,7),
                                                                           initial_ma_short=(1,3), 
                                                                           initial_ma_long=(6,7), 
                                                                           initial_profit=(200,200),
                                                                           initial_book_value=(10000, 10000), 
                                                                           initial_stock_amount=400, 
                                                                           order_expiration_time=120
                                                                          )
    record_state_variables(cur, seed, agents, SIMULATIONTIME, firms, stocks, order_books)
    conn.commit()

In [5]:
cur.close()